In [821]:
# from google.colab import drive
# drive.mount('/content/drive')

In [822]:
import numpy as np
import pandas as pd
import json

In [823]:
Path1='/content/drive/MyDrive/FYP/Images/21_foam_brick/150right/2022-08-11-220931_pv.txt'
Path2="/content/drive/MyDrive/FYP/Images/21_foam_brick/150right/scene_camera.json"
Parameters = pd.read_csv(Path1, sep=' ', header = None)
data = Parameters.to_numpy()

In [824]:
def PNS(data):
  # change the data format to numbers
  # pns: principle points and image size
  pns = ','.join(data[0])
  pns = np.fromstring(pns,dtype=np.float, sep=',' )
  ox = pns[0]
  oy = pns[1]
  im_w = pns[2]
  im_h = pns[3]
  return ox, oy, im_w, im_h

In [825]:
# delete outliers of focal length
def DFL(data):
    if len(data)>41:   
      for i in range(20):
        data = np.delete(data, 1, axis=None)

      for j in range(20):
        data = np.delete(data, len(data)-1, axis=None)
    print(len(data))
    return data

In [826]:
def FRT(data):
    inx=len(data)/2
    inx = int(inx)
    print(inx)
    fcp = data[inx]
    fcp = np.fromstring(fcp,dtype=np.float, sep=',' )
    fx = fcp[1]
    fy = fcp[2]
    camera_pose = fcp[3:]
    camera_pose = camera_pose.reshape((4,4))
    return fx,fy,camera_pose

In [827]:
def RT(camera_pose):
    extrinsic = np.linalg.inv(camera_pose)
    print(extrinsic)
    R = extrinsic[0:3,0:3]
    print(R)
    T = 1000*extrinsic[0:3,3]
    print(T)
    return R,T

In [828]:
def extract_params(data, Path2):
    '''
    input:
    pns:principle points and image size
    fcp: focal legth and camera pose

    output:
    ox:
    oy: Principle points
    R: Rotation matrix
    T: TRanslation vector
    fx:
    fy: focal legth
    '''
    ox, oy, im_w, im_h = PNS(data)
    data = DFL(data)
    fx,fy,camera_pose = FRT(data)

    R,T = RT(camera_pose)
    R = R.flatten()
    
    params = {
        "1" : {"cam_K":[fx.tolist(), 0.0 , ox.tolist(), 0.0, fy.tolist(), oy.tolist(), 0.0, 0.0, 1.0],
               "cam_R_w2c":R.tolist(),
               "cam_t_w2c": T.tolist(),
               "depth_scale": 0.1
               }
    }
    # print(type(params))
    with open(Path2, "w+") as f:
        json.dump(params, f)

    return params


In [829]:
scene_camera = extract_params(data, Path2)
print(scene_camera)

108
54
[[ 9.96569081e-01 -7.46917477e-02  3.56119278e-02  3.02331563e-03]
 [ 8.27477214e-02  8.98756078e-01 -4.30566560e-01 -1.27611375e-01]
 [ 1.52710594e-04  4.32036008e-01  9.01854733e-01  1.49866425e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[ 9.96569081e-01 -7.46917477e-02  3.56119278e-02]
 [ 8.27477214e-02  8.98756078e-01 -4.30566560e-01]
 [ 1.52710594e-04  4.32036008e-01  9.01854733e-01]]
[   3.02331563 -127.61137459  149.86642476]
{'1': {'cam_K': [586.156, 0.0, 373.254, 0.0, 585.632, 201.305, 0.0, 0.0, 1.0], 'cam_R_w2c': [0.996569081418413, -0.07469174774598568, 0.03561192781751323, 0.08274772139586485, 0.8987560779473417, -0.4305665602868064, 0.000152710594481762, 0.4320360078780295, 0.901854732859886], 'cam_t_w2c': [3.023315632174099, -127.61137459433911, 149.86642476330738], 'depth_scale': 0.1}}


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  
